In [3]:
import io
# import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
# from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import tensorflow as tf
#print(check_output(["ls", "input"]).decode("utf8"))
from time import time 

%matplotlib inline

c:\users\admin\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
tfrecords_filename = 'input/tfrecord/img_{}.tfrecords'
opts = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.ZLIB)

In [7]:
filenameQ = tf.train.string_input_producer(['validation-00001-of-00128'],num_epochs=None)
 
    # object to read records
recordReader = tf.TFRecordReader()

    # read the full set of features for a single example 
key, fullExample = recordReader.read(filenameQ)

In [17]:
height=32
width=32


#number of classes is 2 (squares and triangles)
nClass=1000

#simple model (set to True) or convolutional neural network (set to False)
simpleModel=True

def getImage(filename):
    # convert filenames to a queue for an input pipeline.
    filenameQ = tf.train.string_input_producer([filename],num_epochs=None)
 
    # object to read records
    recordReader = tf.TFRecordReader()

    # read the full set of features for a single example 
    key, fullExample = recordReader.read(filenameQ)

    # parse the full example into its' component features.
    features = tf.parse_single_example(
        fullExample,
        features={
            'image/height': tf.FixedLenFeature([], tf.int64),
            'image/width': tf.FixedLenFeature([], tf.int64),
            'image/colorspace': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/channels':  tf.FixedLenFeature([], tf.int64),            
            'image/class/label': tf.FixedLenFeature([],tf.int64),
            'image/class/text': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/format': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/filename': tf.FixedLenFeature([], dtype=tf.string,default_value=''),
            'image/encoded': tf.FixedLenFeature([], dtype=tf.string, default_value='')
        })


    # now we are going to manipulate the label and image features

    label = features['image/class/label']
    image_buffer = features['image/encoded']

    # Decode the jpeg
    with tf.name_scope('decode_jpeg',[image_buffer], None):
        # decode
        image = tf.image.decode_jpeg(image_buffer, channels=3)
    
        # and convert to single precision data type
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)


    # cast image into a single array, where each element corresponds to the greyscale
    # value of a single pixel. 
    # the "1-.." part inverts the image, so that the background is black.

    image=tf.reshape(1-tf.image.rgb_to_grayscale(image),[height*width])

    # re-define label as a "one-hot" vector 
    # it will be [0,1] or [1,0] here. 
    # This approach can easily be extended to more classes.
    label=tf.stack(tf.one_hot(label-1, nClass))

    return label, image

In [23]:
vlabel, vimage = getImage('validation-00001-of-00128')

In [24]:
vimageBatch, vlabelBatch = tf.train.shuffle_batch(
    [vimage, vlabel], batch_size=100,
    capacity=2000,
    min_after_dequeue=1000)

In [26]:
vlabelBatch

<tf.Tensor 'shuffle_batch:1' shape=(100, 1000) dtype=float32>

In [42]:
import tensorflow as tf

for example in tf.python_io.tf_record_iterator("validation-00001-of-00128"):
    result = tf.train.Example.FromString(example)
print(result)

features {
  feature {
    key: "image/channels"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "image/class/label"
    value {
      int64_list {
        value: 621
      }
    }
  }
  feature {
    key: "image/class/synset"
    value {
      bytes_list {
        value: "n03642806"
      }
    }
  }
  feature {
    key: "image/class/text"
    value {
      bytes_list {
        value: "laptop, laptop computer"
      }
    }
  }
  feature {
    key: "image/colorspace"
    value {
      bytes_list {
        value: "RGB"
      }
    }
  }
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\000H\000H\000\000\377\333\000C\000\001\001\001\001\001\001\001\001\001\001\001\001\001\002\002\003\002\002\002\002\002\004\003\003\002\003\005\004\005\005\005\004\004\004\005\006\007\006\005\005\007\006\004\004\006\t\006\007\010\010\010\010\010\005\006\t\n\t\010\n\007\010\010\010\377\333\

In [22]:
def read_and_decode(filename_queue):
    filename_queue = tf.train.string_input_producer([filename_queue], num_epochs = 1)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
    serialized_example,
      # Defaults are not specified since both keys are required.
      features={
          'height': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'depth': tf.FixedLenFeature([], tf.int64),
          'image_raw': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),
      })
    
    
    height = tf.cast(features['height'], tf.int32) 
    width = tf.cast(features['width'], tf.int32) 
    depth = tf.cast(features['depth'], tf.int32) 
    image_shape = tf.stack([height, width, depth]) 
    image = tf.decode_raw(features['image_raw'], tf.uint8) 
    image = tf.reshape(image, image_shape)
    
    print('height:',height)
    print('width:',width)
    print('depth:',depth)
    print('image_shape:',image_shape)
    print('image:',image)
    

  # Convert from a scalar string tensor (whose single string has
  # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
  # [mnist.IMAGE_PIXELS].
#   image = tf.decode_raw(features['image_raw'], tf.uint8)
#   image.set_shape([sess.run(features['height']),sess.run(features['width']),sess.run(features['depth'])])

In [23]:
read_and_decode('validation-00001-of-00128')


height: Tensor("Cast_3:0", shape=(), dtype=int32)
width: Tensor("Cast_4:0", shape=(), dtype=int32)
depth: Tensor("Cast_5:0", shape=(), dtype=int32)
image_shape: Tensor("stack_1:0", shape=(3,), dtype=int32)
image: Tensor("Reshape_1:0", shape=(?, ?, ?), dtype=uint8)


In [46]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# data_path = 'validation-00001-of-00128'  # address to save the hdf5 file
data_path = 'validation-00001-of-00128'  # address to save the hdf5 file
with tf.Session() as sess:

  
    colorspace = 'RGB'
    channels = 3
    image_format = 'JPEG'

    feature = {'image': tf.FixedLenFeature([], tf.string),
               'label': tf.FixedLenFeature([], tf.int64),
              'height': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'depth': tf.FixedLenFeature([], tf.int64),
          'image_raw': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=1)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image'], tf.float32)
    
    # Cast label data into int32
    label = tf.cast(features['label'], tf.int32)
    height = tf.cast(features['height'], tf.int32) 
    width = tf.cast(features['width'], tf.int32) 
    depth = tf.cast(features['depth'], tf.int32) 
    image_shape = tf.stack([height, width, depth]) 
    image = tf.decode_raw(features['image_raw'], tf.uint8) 
#     image = tf.reshape(image, image_shape)
    
    # Reshape image data into the original shape
    image = tf.reshape(image, [224, 224, 3])
    
    # Any preprocessing here ...
    
    # Creates batches by randomly shuffling tensors
    images, labels = tf.train.shuffle_batch([image, label], batch_size=10, capacity=30, num_threads=1, min_after_dequeue=10)
    
    


ValueError: Shapes (2,) and () are not compatible

In [38]:
images

<tf.Tensor 'shuffle_batch_17:0' shape=(10, 224, 224, 3) dtype=uint8>